# Multi Train Gradient Update

## Importing Libraries

In [1]:
from typing import Dict
import threading

import gym
import numpy as np
import torch as th

from stable_baselines3 import A2C as ALGO
from stable_baselines3.common.evaluation import evaluate_policy

## Init. ENV and Model

In [2]:
env = gym.make('CartPole-v1')
model = ALGO(
    "MlpPolicy",
    env
)

model_trained_1 = ALGO(
    "MlpPolicy",
    env
)

model_trained_2 = ALGO(
    "MlpPolicy",
    env
)

## Functions to Evaluate Model and Train Model within Thread

In [3]:
def evaluate(model, env, message = '', verbose = 0):
    fitnesses = []
    iterations = 10
    for i in range(iterations):
        fitness, _ = evaluate_policy(model, env)
        if verbose == 1:
            print(i, fitness, end=" ")
        fitnesses.append(fitness)

    mean_fitness = np.mean(sorted(fitnesses))
    print(f'Type {message} Mean reward: {mean_fitness}')

In [4]:
def train(model, timesteps):
    # print('Starting Training')
    model.learn(total_timesteps=timesteps)
    # print('Completed Training')

## Initial Evaluation

In [5]:
model_trained_1.set_parameters(model.get_parameters())
model_trained_2.set_parameters(model.get_parameters())

evaluate(model, env)
evaluate(model_trained_1, env)
evaluate(model_trained_2, env)

/home/arnab/.miniconda3/envs/pydrl/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Type  Mean reward: 9.639999999999999
Type  Mean reward: 9.459999999999999
Type  Mean reward: 9.629999999999999


## Train for 1K Steps and Evaluate

In [6]:
# Train MT Model 1
t1 = threading.Thread(target=train, args=(model_trained_1, 10_00))

# Train MT Model 2
t2 = threading.Thread(target=train, args=(model_trained_2, 10_00))

# starting thread
t1.start()
t2.start()

# wait until thread is completely executed
t1.join()
t2.join()


# model_trained.learn(total_timesteps=10_00)
evaluate(model_trained_1, env)
evaluate(model_trained_2, env)
evaluate(model, env)

Type  Mean reward: 152.9
Type  Mean reward: 46.11
Type  Mean reward: 9.620000000000001


## Apply Gradient and Evaluate

In [7]:
# For Trained Model 1
state_dict = model.policy.state_dict()
optim_dict = model_trained_1.policy.optimizer.param_groups[0]['params']
optim_alpha = model.policy.optimizer.param_groups[0]['alpha']

optim_index = 0
for key, value in state_dict.items():
    # print(key)
    state_dict[key].add_(optim_alpha, optim_dict[optim_index])
    optim_index += 1

model.policy.load_state_dict(state_dict)

# For Trained Model 2
state_dict = model.policy.state_dict()
optim_dict = model_trained_2.policy.optimizer.param_groups[0]['params']
optim_alpha = model.policy.optimizer.param_groups[0]['alpha']

optim_index = 0
for key, value in state_dict.items():
    # print(key)
    state_dict[key].add_(optim_alpha, optim_dict[optim_index])
    optim_index += 1

model.policy.load_state_dict(state_dict)
model_trained_1.policy.load_state_dict(state_dict)
model_trained_2.policy.load_state_dict(state_dict)


evaluate(model, env)

/home/arnab/.miniconda3/envs/pydrl/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1025.)
  if __name__ == '__main__':


Type  Mean reward: 119.2


In [8]:
for i in range(100):
    print('Train Iter: ', i, end="| ")

    # Train MT Model 1
    t1 = threading.Thread(target=train, args=(model_trained_1, 10_0))

    # Train MT Model 2
    t2 = threading.Thread(target=train, args=(model_trained_2, 10_0))

    # starting thread
    t1.start()
    t2.start()

    # wait until thread is completely executed
    t1.join()
    t2.join()

    # evaluate(model_trained_1, env, 'Trained Model 1', verbose=1)
    # evaluate(model_trained_2, env, 'Trained Model 2', verbose=1)
    # evaluate(model, env, 'Initial Model', verbose=1)

    # For Trained Model 1
    state_dict = model.policy.state_dict()
    optim_dict = model_trained_1.policy.optimizer.param_groups[0]['params']
    optim_alpha = model.policy.optimizer.param_groups[0]['alpha']

    optim_index = 0
    for key, value in state_dict.items():
        # print(key)
        state_dict[key].add_(optim_alpha, optim_dict[optim_index])
        optim_index += 1

    model.policy.load_state_dict(state_dict)

    # For Trained Model 2
    state_dict = model.policy.state_dict()
    optim_dict = model_trained_2.policy.optimizer.param_groups[0]['params']
    optim_alpha = model.policy.optimizer.param_groups[0]['alpha']

    optim_index = 0
    for key, value in state_dict.items():
        # print(key)
        state_dict[key].add_(optim_alpha, optim_dict[optim_index])
        optim_index += 1

    model.policy.load_state_dict(state_dict)


    evaluate(model, env, 'Updated Model', verbose=0)

Train Iter:  0| Type Updated Model Mean reward: 150.29000000000002
Train Iter:  1| Type Updated Model Mean reward: 150.32
Train Iter:  2| Type Updated Model Mean reward: 172.71999999999997
Train Iter:  3| Type Updated Model Mean reward: 129.89000000000001
Train Iter:  4| Type Updated Model Mean reward: 171.75
Train Iter:  5| Type Updated Model Mean reward: 168.24
Train Iter:  6| Type Updated Model Mean reward: 188.44
Train Iter:  7| Type Updated Model Mean reward: 166.49
Train Iter:  8| Type Updated Model Mean reward: 168.60999999999999
Train Iter:  9| Type Updated Model Mean reward: 184.83999999999997
Train Iter:  10| Type Updated Model Mean reward: 191.07
Train Iter:  11| Type Updated Model Mean reward: 181.01
Train Iter:  12| Type Updated Model Mean reward: 187.98999999999998
Train Iter:  13| Type Updated Model Mean reward: 175.65
Train Iter:  14| Type Updated Model Mean reward: 165.59
Train Iter:  15| Type Updated Model Mean reward: 170.51999999999998
Train Iter:  16| Type Updated 

In [ ]:
model.get_parameters()

In [ ]:
model.save('a2c_lunar_multiproc')

In [ ]:
# Exporting Params as JSON
## Function to Convert Params Dict to Flattened List
def flatten_list(params):
    """
    :param params: (dict)
    :return: (np.ndarray)
    """
    params_ = {}
    for key in params.keys():
        params_[key] = params[key].tolist()
    return params_
## Write Parameters to JSON File
import json

all_params = model.get_parameters()
pol_params = flatten_list(all_params['policy'])

all_params['policy'] = pol_params

with open('a2c_lunar_multiproc.json', 'w') as f:
    json.dump(all_params, f, indent='\t')

In [ ]:
model_loaded = ALGO(
    "MlpPolicy",
    env
)

evaluate(model_loaded,env, verbose=1)

new_params = all_params
loaded_pol_params = new_params['policy']
for key in loaded_pol_params.keys():
    loaded_pol_params[key] = th.tensor(loaded_pol_params[key])

new_params['policy'] = loaded_pol_params

model_loaded.set_parameters(new_params)

In [ ]:
env.reset()
evaluate(model_loaded,env, verbose=1)